In [2]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config
from torch.utils.data import DataLoader
from transformers import ViTImageProcessor, ViTModel, ViTConfig, CLIPProcessor, CLIPModel
from sentence_transformers import SentenceTransformer, models
from transformers import AutoImageProcessor, AutoModel
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

import torch, random
from tqdm import tqdm
import sys
import re

gpu = 'cuda:0'
device = torch.device(gpu)
torch.cuda.empty_cache()

/home/raiymbek/miniconda3/envs/vlm_proper/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def cluster_text(model, dataset):
    source_base_large = torch.load(f"{dataset}_{model}_text.pt", map_location=gpu)
    clustering_matrix = source_base_large.cpu().numpy()
    kmeans = KMeans(n_clusters=320, random_state=0).fit(clustering_matrix)
    
    cos_sims = cosine_similarity(kmeans.cluster_centers_, clustering_matrix)
    closest_indices = np.argsort(cos_sims, axis=1)[:,-1:]
    closest_indices = list(set(closest_indices.reshape(-1)))
    choose = closest_indices
    
    torch.save(choose, f'{dataset}_{model}_text_cluster.pt')

In [34]:
for model in ["allroberta", "clip", ]:
    for dataset in ["coco", "nocaps"]:
        cluster_text(model, dataset)

/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to

In [3]:
def cluster_img(model, dataset):
    source_base_large = torch.load(f"{dataset}_{model}_img.pt", map_location=gpu)
    clustering_matrix = source_base_large.cpu().numpy()
    kmeans = KMeans(n_clusters=320, random_state=0).fit(clustering_matrix)
    
    cos_sims = cosine_similarity(kmeans.cluster_centers_, clustering_matrix)
    closest_indices = np.argsort(cos_sims, axis=1)[:,-1:]
    closest_indices = list(set(closest_indices.reshape(-1)))
    choose = closest_indices
    
    torch.save(choose, f'{dataset}_{model}_img_cluster.pt')

In [14]:
for model in ["vit"]:
    for dataset in ["coco"]:
        cluster_img(model, dataset)

/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [5]:
for model in ["sam_embed"]:
    for dataset in ["coco"]:
        cluster_img(model, dataset)

/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (320). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [3]:
for model in ["detr_resnet_101_encoder", "detr_resnet_101_decoder", "detr_resnet_50_encoder", "detr_resnet_50_decoder", "sam", "segformer", "dpt"]:
    for dataset in ["coco"]:
        cluster_img(model, dataset)

/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to

In [21]:
source_base_large = torch.load(f"coco_text_no_transform.pt", map_location=gpu)
target_base_large = torch.load(f"coco_val_clip_vit_large_patch14_img.pt", map_location=gpu)

clustering_matrix = source_base_large.cpu().numpy()
kmeans = KMeans(n_clusters=320, random_state=0).fit(clustering_matrix)

cos_sims = cosine_similarity(kmeans.cluster_centers_, clustering_matrix)
closest_indices = np.argsort(cos_sims, axis=1)[:,-1:]
closest_indices = list(set(closest_indices.reshape(-1)))
choose = closest_indices

source_base = source_base_large[choose]
target_base = target_base_large[choose]

torch.save(source_base_large, f'coco_text_no_transform_320_c1.pt')
torch.save(target_base_large, f'coco_val_clip_vit_large_patch14_img_320_c1.pt')

/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [16]:
source_base_large = torch.load(f"coco_text_no_transform.pt", map_location=gpu)
target_base_large = torch.load(f"convnext_coco_val.pt", map_location=gpu)

clustering_matrix = source_base_large.cpu().numpy()
kmeans = KMeans(n_clusters=320, random_state=0).fit(clustering_matrix)

cos_sims = cosine_similarity(kmeans.cluster_centers_, clustering_matrix)
closest_indices = np.argsort(cos_sims, axis=1)[:,-1:]
closest_indices = list(set(closest_indices.reshape(-1)))
choose = closest_indices

source_base = source_base_large[choose]
target_base = target_base_large[choose]

torch.save(source_base_large, f'coco_text_no_transform_320_c1.pt')
torch.save(target_base_large, f'convnext_coco_val_320_c1.pt')

/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [20]:
source_base_large = torch.load(f"coco_text_no_transform.pt", map_location=gpu)
target_base_large = torch.load(f"coco_val_clip_vit_large_patch14_img.pt", map_location=gpu)

clustering_matrix = target_base_large.cpu().numpy()
kmeans = KMeans(n_clusters=320, random_state=0).fit(clustering_matrix)

cos_sims = cosine_similarity(kmeans.cluster_centers_, clustering_matrix)
closest_indices = np.argsort(cos_sims, axis=1)[:,-1:]
closest_indices = list(set(closest_indices.reshape(-1)))
choose = closest_indices

source_base = source_base_large[choose]
target_base = target_base_large[choose]

torch.save(source_base_large, f'coco_text_no_transform_320_c2.pt')
torch.save(target_base_large, f'coco_val_clip_vit_large_patch14_img_320_c2.pt')

/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [18]:
source_base_large = torch.load(f"coco_text_no_transform.pt", map_location=gpu)
target_base_large = torch.load(f"convnext_coco_val.pt", map_location=gpu)

clustering_matrix = target_base_large.cpu().numpy()
kmeans = KMeans(n_clusters=320, random_state=0).fit(clustering_matrix)

cos_sims = cosine_similarity(kmeans.cluster_centers_, clustering_matrix)
closest_indices = np.argsort(cos_sims, axis=1)[:,-1:]
closest_indices = list(set(closest_indices.reshape(-1)))
choose = closest_indices

source_base = source_base_large[choose]
target_base = target_base_large[choose]

torch.save(source_base_large, f'coco_text_no_transform_320_c2.pt')
torch.save(target_base_large, f'convnext_coco_val_320_c2.pt')

/home/raiymbek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
torch.save(source_base_large, f'coco_val_clip_vit_large_patch14_img_320_c1.pt')
torch.save(target_base_large, f'nocap_val_clip_vit_large_patch14_img_320_c1.pt.pt')

In [ ]:
source_base_large = torch.load(f"coco_val_clip_vit_large_patch14_img.pt", map_location=gpu)
target_base_large = torch.load(f"coco_text_no_transform.pt", map_location=gpu)
clustering_matrix = source_base_large.cpu().numpy()
kmeans = KMeans(n_clusters=320, random_state=0).fit(clustering_matrix)

cos_sims = cosine_similarity(kmeans.cluster_centers_, clustering_matrix)
closest_indices = np.argsort(cos_sims, axis=1)[:,-1:]
closest_indices = list(set(closest_indices.reshape(-1)))
choose = closest_indices

source_base = source_base_large[choose]
target_base = target_base_large[choose]

torch.save(source_base_large, f'coco_val_clip_vit_large_patch14_img_320_c1.pt')
torch.save(target_base_large, f'nocap_val_clip_vit_large_patch14_img_320_c1.pt.pt')